# Realworld Experiments

In [1]:
import os
import copy
import dask
import pickle
import pandas as pd
from lifelines import CoxPHFitter
from lifelines.statistics import logrank_test
from oldsrc.skfibers.fibers import FIBERS as FIBERS1
from src.skfibers.fibers import FIBERS as FIBERS2
from src.skfibers.methods.data_handling import prepare_data
from sklearn.metrics import accuracy_score
from dask.distributed import Client
from dask_jobqueue import SLURMCluster, LSFCluster, SGECluster

## Common Functions

In [2]:
def make_obj_list(fibers, param):  
    obj_list = list()
    for i in range(1):
        fibers.random_seed = i+1
        obj_list.append((copy.deepcopy(fibers), param))
    return obj_list

In [3]:
# Define key column names to be used in analysis
outcome_label = "graftyrs"
censor_label = "grf_fail"
amino_acids = ['A1', 'A10', 'A100', 'A101', 'A102', 'A103', 'A104', 'A105', 'A106', 'A107', 'A108', 'A109', 'A11', 'A110', 'A111', 'A112', 'A113', 'A114', 'A115', 'A116', 'A117', 'A118', 'A119', 'A12', 'A120', 'A121', 'A122', 'A123', 'A124', 'A125', 'A126', 'A127', 'A128', 'A129', 'A13', 'A130', 'A131', 'A132', 'A133', 'A134', 'A135', 'A136', 'A137', 'A138', 'A139', 'A14', 'A140', 'A141', 'A142', 'A143', 'A144', 'A145', 'A146', 'A147', 'A148', 'A149', 'A15', 'A150', 'A151', 'A152', 'A153', 'A154', 'A155', 'A156', 'A157', 'A158', 'A159', 'A16', 'A160', 'A161', 'A162', 'A163', 'A164', 'A165', 'A166', 'A167', 'A168', 'A169', 'A17', 'A170', 'A171', 'A172', 'A173', 'A174', 'A175', 'A176', 'A177', 'A178', 'A179', 'A18', 'A180', 'A181', 'A182', 'A183', 'A184', 'A185', 'A186', 'A187', 'A188', 'A189', 'A19', 'A190', 'A191', 'A192', 'A193', 'A194', 'A195', 'A196', 'A197', 'A198', 'A199', 'A2', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A3', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A4', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A5', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A6', 'A60', 'A61', 'A62', 'A63', 'A64', 'A65', 'A66', 'A67', 'A68', 'A69', 'A7', 'A70', 'A71', 'A72', 'A73', 'A74', 'A75', 'A76', 'A77', 'A78', 'A79', 'A8', 'A80', 'A81', 'A82', 'A83', 'A84', 'A85', 'A86', 'A87', 'A88', 'A89', 'A9', 'A90', 'A91', 'A92', 'A93', 'A94', 'A95', 'A96', 'A97', 'A98', 'A99', 'B1', 'B10', 'B100', 'B101', 'B102', 'B103', 'B104', 'B105', 'B106', 'B107', 'B108', 'B109', 'B11', 'B110', 'B111', 'B112', 'B113', 'B114', 'B115', 'B116', 'B117', 'B118', 'B119', 'B12', 'B120', 'B121', 'B122', 'B123', 'B124', 'B125', 'B126', 'B127', 'B128', 'B129', 'B13', 'B130', 'B131', 'B132', 'B133', 'B134', 'B135', 'B136', 'B137', 'B138', 'B139', 'B14', 'B140', 'B141', 'B142', 'B143', 'B144', 'B145', 'B146', 'B147', 'B148', 'B149', 'B15', 'B150', 'B151', 'B152', 'B153', 'B154', 'B155', 'B156', 'B157', 'B158', 'B159', 'B16', 'B160', 'B161', 'B162', 'B163', 'B164', 'B165', 'B166', 'B167', 'B168', 'B169', 'B17', 'B170', 'B171', 'B172', 'B173', 'B174', 'B175', 'B176', 'B177', 'B178', 'B179', 'B18', 'B180', 'B181', 'B182', 'B183', 'B184', 'B185', 'B186', 'B187', 'B188', 'B189', 'B19', 'B190', 'B191', 'B192', 'B193', 'B194', 'B2', 'B20', 'B21', 'B22', 'B23', 'B24', 'B25', 'B26', 'B27', 'B28', 'B29', 'B3', 'B30', 'B31', 'B32', 'B33', 'B34', 'B35', 'B36', 'B37', 'B38', 'B39', 'B4', 'B40', 'B41', 'B42', 'B43', 'B44', 'B45', 'B46', 'B47', 'B48', 'B49', 'B5', 'B50', 'B51', 'B52', 'B53', 'B54', 'B55', 'B56', 'B57', 'B58', 'B59', 'B6', 'B60', 'B61', 'B62', 'B63', 'B64', 'B65', 'B66', 'B67', 'B68', 'B69', 'B7', 'B70', 'B71', 'B72', 'B73', 'B74', 'B75', 'B76', 'B77', 'B78', 'B79', 'B8', 'B80', 'B81', 'B82', 'B83', 'B84', 'B85', 'B86', 'B87', 'B88', 'B89', 'B9', 'B90', 'B91', 'B92', 'B93', 'B94', 'B95', 'B96', 'B97', 'B98', 'B99', 'C1', 'C10', 'C100', 'C101', 'C102', 'C103', 'C104', 'C105', 'C106', 'C107', 'C108', 'C109', 'C11', 'C110', 'C111', 'C112', 'C113', 'C114', 'C115', 'C116', 'C117', 'C118', 'C119', 'C12', 'C120', 'C121', 'C122', 'C123', 'C124', 'C125', 'C126', 'C127', 'C128', 'C129', 'C13', 'C130', 'C131', 'C132', 'C133', 'C134', 'C135', 'C136', 'C137', 'C138', 'C139', 'C14', 'C140', 'C141', 'C142', 'C143', 'C144', 'C145', 'C146', 'C147', 'C148', 'C149', 'C15', 'C150', 'C151', 'C152', 'C153', 'C154', 'C155', 'C156', 'C157', 'C158', 'C159', 'C16', 'C160', 'C161', 'C162', 'C163', 'C164', 'C165', 'C166', 'C167', 'C168', 'C169', 'C17', 'C170', 'C171', 'C172', 'C173', 'C174', 'C175', 'C176', 'C177', 'C178', 'C179', 'C18', 'C180', 'C181', 'C182', 'C183', 'C184', 'C185', 'C186', 'C187', 'C188', 'C189', 'C19', 'C190', 'C191', 'C192', 'C193', 'C194', 'C195', 'C196', 'C197', 'C198', 'C199', 'C2', 'C20', 'C200', 'C201', 'C202', 'C203', 'C204', 'C205', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C3', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C4', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C5', 'C50', 'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C6', 'C60', 'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C7', 'C70', 'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79', 'C8', 'C80', 'C81', 'C82', 'C83', 'C84', 'C85', 'C86', 'C87', 'C88', 'C89', 'C9', 'C90', 'C91', 'C92', 'C93', 'C94', 'C95', 'C96', 'C97', 'C98', 'C99', 'DQB11', 'DQB110', 'DQB111', 'DQB112', 'DQB113', 'DQB114', 'DQB115', 'DQB116', 'DQB117', 'DQB118', 'DQB119', 'DQB12', 'DQB120', 'DQB121', 'DQB122', 'DQB123', 'DQB124', 'DQB125', 'DQB126', 'DQB127', 'DQB128', 'DQB129', 'DQB13', 'DQB130', 'DQB131', 'DQB132', 'DQB133', 'DQB134', 'DQB135', 'DQB136', 'DQB137', 'DQB138', 'DQB139', 'DQB14', 'DQB140', 'DQB141', 'DQB142', 'DQB143', 'DQB144', 'DQB145', 'DQB146', 'DQB147', 'DQB148', 'DQB149', 'DQB15', 'DQB150', 'DQB151', 'DQB152', 'DQB153', 'DQB154', 'DQB155', 'DQB156', 'DQB157', 'DQB158', 'DQB159', 'DQB16', 'DQB160', 'DQB161', 'DQB162', 'DQB163', 'DQB164', 'DQB165', 'DQB166', 'DQB167', 'DQB168', 'DQB169', 'DQB17', 'DQB170', 'DQB171', 'DQB172', 'DQB173', 'DQB174', 'DQB175', 'DQB176', 'DQB177', 'DQB178', 'DQB179', 'DQB18', 'DQB180', 'DQB181', 'DQB182', 'DQB183', 'DQB184', 'DQB185', 'DQB186', 'DQB187', 'DQB188', 'DQB189', 'DQB19', 'DQB190', 'DQB191', 'DQB192', 'DQB193', 'DQB194', 'DRB11', 'DRB110', 'DRB111', 'DRB112', 'DRB113', 'DRB114', 'DRB115', 'DRB116', 'DRB117', 'DRB118', 'DRB119', 'DRB12', 'DRB120', 'DRB121', 'DRB122', 'DRB123', 'DRB124', 'DRB125', 'DRB126', 'DRB127', 'DRB128', 'DRB129', 'DRB13', 'DRB130', 'DRB131', 'DRB132', 'DRB133', 'DRB134', 'DRB135', 'DRB136', 'DRB137', 'DRB138', 'DRB139', 'DRB14', 'DRB140', 'DRB141', 'DRB142', 'DRB143', 'DRB144', 'DRB145', 'DRB146', 'DRB147', 'DRB148', 'DRB149', 'DRB15', 'DRB150', 'DRB151', 'DRB152', 'DRB153', 'DRB154', 'DRB155', 'DRB156', 'DRB157', 'DRB158', 'DRB159', 'DRB16', 'DRB160', 'DRB161', 'DRB162', 'DRB163', 'DRB164', 'DRB165', 'DRB166', 'DRB167', 'DRB168', 'DRB169', 'DRB17', 'DRB170', 'DRB171', 'DRB172', 'DRB173', 'DRB174', 'DRB175', 'DRB176', 'DRB177', 'DRB178', 'DRB179', 'DRB18', 'DRB180', 'DRB181', 'DRB182', 'DRB183', 'DRB184', 'DRB185', 'DRB186', 'DRB187', 'DRB188', 'DRB189', 'DRB19', 'DRB190', 'DRB191', 'DRB192', 'DRB193', 'DRB194']
covariates = ['shared', 'DCD', 'DON_AGE', 'donage_slope_ge18', 'dcadcodanox', 'dcadcodcva', 'dcadcodcnst', 'dcadcodoth', 'don_cmv_negative', 
            'don_htn_0c', 'ln_don_wgt_kg_0c', 'ln_don_wgt_kg_0c_s55', 'don_ecd', 'age_ecd', 'yearslice', 'REC_AGE_AT_TX', 
            'rec_age_spline_35', 'rec_age_spline_50', 'rec_age_spline_65', 'diab_noted', 'age_diab', 'dm_can_age_spline_50', 
            'can_dgn_htn_ndm', 'can_dgn_pk_ndm', 'can_dgn_gd_ndm', 'rec_prev_ki_tx', 'rec_prev_ki_tx_dm', 'rbmi_0c', 'rbmi_miss', 
            'rbmi_gt_20', 'rbmi_DM', 'rbmi_gt_20_DM', 'ln_c_hd_m', 'ln_c_hd_0c', 'ln_c_hd_m_ptx', 'PKPRA_MS', 'PKPRA_1080', 
            'PKPRA_GE80', 'hispanic', 'CAN_RACE_BLACK', 'CAN_RACE_asian', 'CAN_RACE_WHITE', 'mm0', 'mmDR0', 'mmDR1', 'mmA0', 'mmA1', 
            'mmB0', 'mmB1', 'mmC0', 'mmC1', 'mmDQ0', 'mmDQ1']


def read_data(data_path = 'PPSNDatasets/'+ 'realworld_imp1.csv'):
    data = pd.read_csv(data_path)
    print("Shape of Raw DataFrame:", data.shape)
    missing_sum = data.isna().sum().sum()
    print("Sum of raw data missing values:", missing_sum)

    features = amino_acids + covariates + [outcome_label] + [censor_label]
    print(str(len(features))+ " Total Columns")
    print(str(len(amino_acids))+" AAs")
    print(str(len(covariates))+ " Covariates")

    data = data[features]
    print("Shape of New DataFrame:", data.shape)
    missing_sum = data.isna().sum().sum()
    print("Sum of data missing values:", missing_sum)

    #Debugging
    # Create a new DataFrame with 100 random rows
    #data = data.sample(n=1000)
    #Covariate examination
    column_types_count = data.dtypes.value_counts()
    print(column_types_count)
    object_columns = data.columns[data.dtypes == 'object']
    print(object_columns)
    num_categories = data['REC_AGE_AT_TX'].unique()
    print(num_categories)
    #Process covariates as needed
    cat_columns = data.select_dtypes(['object']).columns
    data[cat_columns] = data[cat_columns].apply(lambda x: pd.factorize(x)[0])   #IMPORTANT - this encoding is temporary to replicate paper analysis.  This must be fixed so that age ranges are ordinally encoded.
    num_categories = data['REC_AGE_AT_TX'].unique()
    print(num_categories)
    filter_all_rare = True
    rare_freq = 0.01
    if filter_all_rare:
        #Filter out rare AAs (<1%)
        # Calculate the percentage of occurrences greater than 0 for each column
        percentages = data.loc[:,amino_acids].apply(lambda x: (x > 0).mean())
        # print(percentages)
        columns_to_remove = percentages[percentages < rare_freq].index.tolist()
        data = data.drop(columns=columns_to_remove)
        # print(data.shape)
    else:
        #Filter out invariant AAs
        # Calculate the percentage of occurrences greater than 0 for each column
        percentages = data.loc[:,amino_acids].apply(lambda x: (x > 0).mean())
        # print(percentages)
        columns_to_remove = percentages[percentages == 0.0].index.tolist()
        data = data.drop(columns=columns_to_remove)
        # print(data.shape)
    return data

In [4]:
def get_cluster(cluster_type='SLURM', output_path=".", queue='defq', memory=16):
    client = None
    try:
        if cluster_type == 'SLURM':
            cluster = SLURMCluster(queue=queue,
                                   cores=1,
                                   memory=str(memory) + "G",
                                   walltime="24:00:00",
                                   log_directory=output_path + "/dask_logs/")
            cluster.adapt(maximum_jobs=400)
        elif cluster_type == "LSF":
            cluster = LSFCluster(queue=queue,
                                 cores=1,
                                 mem=memory * 1000000000,
                                 memory=str(memory) + "G",
                                 walltime="24:00",
                                 log_directory=output_path + "/dask_logs/")
            cluster.adapt(maximum_jobs=400)
        elif cluster_type == 'UGE':
            cluster = SGECluster(queue=queue,
                                 cores=1,
                                 memory=str(memory) + "G",
                                 resource_spec="mem_free=" + str(memory) + "G",
                                 walltime="24:00:00",
                                 log_directory=output_path + "/dask_logs/")
            cluster.adapt(maximum_jobs=400)
        elif cluster_type == 'Local':
            c = Client()
            cluster = c.cluster
        else:
            raise Exception("Unknown or Unsupported Cluster Type")
        client = Client(cluster)
    except Exception as e:
        print(e)
        raise Exception("Exception: Unknown Exception")
    print("Running dask-cluster")
    print(client.scheduler_info())
    return client

In [5]:
client = get_cluster('SLURM')

Running dask-cluster
{'type': 'Scheduler', 'id': 'Scheduler-95255752-8554-48db-b1b4-045b891a3d47', 'address': 'tcp://172.21.0.79:44453', 'services': {'dashboard': 37375}, 'started': 1713377864.9714305, 'workers': {}}


/home/bandheyh/common/anaconda3/envs/fibers/lib/python3.8/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37375 instead
  warnings.warn(


## FIBERS 1

In [6]:
root_folder_1 = 'PPSNResults/Realworld1/'
dataset_name = 'realworld_imp1.csv'
experiment_name = 'Goal7'

In [7]:
def param_maker_1(dataset_name, experiment_name):
    param_dict = {
        'root_folder': root_folder_1,
        'dataset_name': dataset_name,
        'experiment_name': experiment_name,
    }
    return param_dict

In [8]:
fibers1 = FIBERS1(label_name="grf_fail", duration_name="graftyrs",
                given_starting_point=False, start_point_feature_list=None, 
                feature_bins_start_point=None, iterations=100,
                set_number_of_bins=50, min_features_per_group=1,
                max_number_of_groups_with_feature=10,
                informative_cutoff=0.2, crossover_probability=0.5,
                mutation_probability=0.1, elitism_parameter=0.1,
                random_seed=42, threshold = 0, evolving_probability = 0.0,
                min_threshold = 0, max_threshold = 5,
                merge_probability = 0.1, adaptable_threshold=True, covariates=covariates,
                scoring_method="log_rank")                      
   

In [9]:
# fibers1.fit(data)

In [10]:
def get_experiment_output_fibers1(fibers, X, y=None, dataset=None, filename=None):
        columns = ["Features in Bin", "Number of P", "Number of R", 
                   "Bin Size", "Pred Ratio", "Birth Iteration",
                   "Iterations to Ideal Solution", 
                   "Log-Rank Score",
                   "Unadjusted HR", "HR CI", "HR P-value", "Runtime",
                   "Count At/Below Threshold",
                   "Count Above Threshold", "Group Ratio", 
                   "Log-Rank p-value", "Threshold", "Accuracy",
                   "Residual",
                   "Residual p-value", 
                   "Dataset Filename"]
  
        if not fibers.hasTrained:
            raise Exception("Model must be trained first")
        
        durations_no, durations_mm, event_observed_no, event_observed_mm, top_bin = fibers.get_duration_event(0)
        results = logrank_test(durations_no, durations_mm, event_observed_A=event_observed_no,
                               event_observed_B=event_observed_mm)
        
        sorted_bin_scores = dict(sorted(fibers.bin_scores.items(), key=lambda item: item[1], reverse=True))
        sorted_bin_list = list(sorted_bin_scores.keys())
        Bin = fibers.bins[sorted_bin_list[0]]
        d_data = X.copy()
        d_data['Bin'] = d_data[Bin].sum(axis=1)
        column_values = d_data['Bin'].to_list()
        for r in range(0, len(column_values)):
            if column_values[r] > 0:
                column_values[r] = 1
        d_data['Bin'] = column_values
        coxmodeldata =  d_data[["Bin", fibers.duration_name, fibers.label_name] + fibers.covariates]
        cph = CoxPHFitter()
        cph.fit(coxmodeldata, fibers.duration_name, event_col=fibers.label_name, show_progress=True)
        summary = cph.summary
        HR = summary['exp(coef)'].iloc[0]
        HR_CI = str(summary['exp(coef) lower 95%'].iloc[0])+'-'+str(summary['exp(coef) upper 95%'].iloc[0])
        HR_p_value = summary['p'].iloc[0]
        pdf = pd.DataFrame([[fibers.bins[top_bin], str(fibers.bins[top_bin]).count('P'), str(fibers.bins[top_bin]).count('R'), 
                             len(fibers.bins[top_bin]), str(fibers.bins[top_bin]).count('P')/len(fibers.bins[top_bin]), None,
                             None if str(fibers.bins[top_bin]).count('P') != 10 else 101,
                             fibers.bin_scores[top_bin], 
                             HR, HR_CI, HR_p_value, fibers.elapsed_time,
                             len(durations_no),
                             len(durations_mm), len(durations_no)/len(durations_mm), 
                             results.p_value, fibers.bins[top_bin].get_threshold(), 
                             None, 
                             None, None, dataset,]],
                           columns=columns).T  # SPHIA
        
        if filename:
            pdf.to_csv(filename)
            
        return pdf

In [11]:
# get_experiment_output_fibers1(fibers1, data)

In [12]:
def runner_fn_fibers1(fibers, params):
    data = read_data('PPSNDatasets/realworld_imp1/CVDatasets/'
                      + params['dataset_name'])
    fibers = fibers.fit(data) 
    experiment_results = get_experiment_output_fibers1(fibers, data, None, params['dataset_name'],
                                               params['root_folder'] + '/' + params['experiment_name'] 
                                               + '/' + params['dataset_name'].split('.')[0] 
                                               + '/models/model_table_' + str(fibers.random_seed) + '.csv')
    with open(params['root_folder'] + '/' + params['experiment_name'] 
              + '/' + params['dataset_name'].split('.')[0] + '/models/' + str(fibers.random_seed), 'wb') as file:
        pickle.dump(fibers, file)
    print(params)
    return experiment_results

In [13]:
datasets = list(sorted(os.listdir('PPSNDatasets/realworld_imp1/CVDatasets')))
param_grid = [("Goal7", dataset, fibers1) for dataset in datasets]
# param_grid

[('Goal7',
  'realworld_imp1_CV_0_Test.csv',
  FIBERS(adaptable_threshold=True,
         covariates=['shared', 'DCD', 'DON_AGE', 'donage_slope_ge18',
                     'dcadcodanox', 'dcadcodcva', 'dcadcodcnst', 'dcadcodoth',
                     'don_cmv_negative', 'don_htn_0c', 'ln_don_wgt_kg_0c',
                     'ln_don_wgt_kg_0c_s55', 'don_ecd', 'age_ecd', 'yearslice',
                     'REC_AGE_AT_TX', 'rec_age_spline_35', 'rec_age_spline_50',
                     'rec_age_spline_65', 'diab_noted', 'age_diab',
                     '...k_ndm',
                     'can_dgn_gd_ndm', 'rec_prev_ki_tx', 'rec_prev_ki_tx_dm',
                     'rbmi_0c', 'rbmi_miss', 'rbmi_gt_20', ...],
         duration_name='graftyrs', elitism_parameter=0.1,
         evolving_probability=0.0, iterations=100, label_name='grf_fail',
         max_number_of_groups_with_feature=10, max_threshold=5,
         merge_probability=0.1, min_features_per_group=1,
         mutation_probability=0.1, ran

In [14]:
# for experiment_name in experiment_list:
#     for dataset_name in dataset_name_list:
DEBUG = True
if DEBUG:
    import shutil
    try:
        shutil.rmtree(root_folder_1)
    except:
        pass
for experiment_name, dataset_name, _ in param_grid:
    try:
        folder = root_folder_1 + '/' + experiment_name + '/' + dataset_name.split('.')[0] + '/'
        os.makedirs(folder)
        os.makedirs(folder + '/models/')
    except FileExistsError:
        folder = root_folder_1 + '/' + experiment_name + '/' + dataset_name.split('.')[0] + '/'
        print("Folder Already Exists:" + folder)
        os.makedirs(folder + '/models/')
        continue

In [15]:
job_list = list()
for experiment_name, dataset_name, fibers in param_grid: 
    job_list.extend(make_obj_list(fibers, param_maker_1(dataset_name, experiment_name)))

In [16]:
results = dask.compute([dask.delayed(runner_fn_fibers1)(fibers_obj, params
                                            ) for fibers_obj, params in job_list])

In [17]:
result_df = pd.concat(results[0], axis=1, ignore_index=False).T

In [18]:
outname = 'experiment_table_f1.csv'


outdir = root_folder_1 + 'Goal7'
if not os.path.exists(outdir):
    os.makedirs(outdir)

fullname = os.path.join(outdir, outname)    

result_df.to_csv(fullname)